## POI function

Obtain the number of parking spots around any point of interest (as tagged on OSM). Returns geopandas dataframes.

In [3]:
# Imports
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def get_osm_data(lor_gdf, tags, crs="EPSG:25833", location="Berlin, Germany"):
    # Load data
    try:
        lor_gdf = gpd.read_file(lor_gdf)
    except:
        print("Can't load LOR file")
    # Obtain coordinates of POIs
    poi = ox.geometries_from_place(location, tags=tags)
    poi_points = poi[poi.geom_type == "Point"]
    
    # Drop unnecessary (debatable) information
    poi = poi_points.dropna(axis=1)
    poi["name"] = poi_points["name"]
    
    # Project to the same coordinate system
    poi = poi.to_crs(crs)
    
    # We look for the intersection of all pois and our Bezirk
    poi_lor = gpd.sjoin(poi, lor_gdf, predicate='intersects')
    
    return poi_lor

In [5]:
def calc_parking_spots(poi_lor, est_parking_spots, agg_func="len", buffer_size=20):
    # Load data
    try:
        est_parking_spots = gpd.read_file(est_parking_spots)
    except:
        print("Can't load parking spots file")
        
    # Define a circular region around each poi
    poi_lor_buffered = poi_lor.copy()
    poi_lor_buffered["geometry"] = poi_lor["geometry"].buffer(buffer_size)
    poi_lor_buffered = poi_lor_buffered.drop(labels=["index_right"], axis=1)
    
    # Intersect parking spots and area
    intersect_parking = gpd.sjoin(poi_lor_buffered, est_parking_spots)
    
    # Calculate parking spots
    intersect_parking["capacity"] = intersect_parking["capacity"].apply(int)
    if agg_func == "len":
        dfpivot = pd.pivot_table(
            intersect_parking,
            index="name_left",
            columns="capacity",
            aggfunc={"capacity": len},
        )
    else:
        dfpivot = pd.pivot_table(
            intersect_parking,
            index="name_left",
            columns="capacity",
            aggfunc={"capacity": np.sum},
        )
    dfpivot.columns = dfpivot.columns.droplevel()
    dfpivot.index.names = ["name"]
    parking_counts = dfpivot.sum(axis=1)
    parking_counts = parking_counts.to_frame("No. parking spots")
    
    # Merge back with the main dataframe
    poi_lor_counts = poi_lor.merge(parking_counts.reset_index(), on="name", how="left")
    poi_lor_counts["No. parking spots"] = poi_lor_counts["No. parking spots"].fillna(0)
    
    return poi_lor_counts, poi_lor_buffered

In [6]:
# Usage:
poi_lor = get_osm_data(
    r"C:\Users\Michl\Documents\GitHub\data-ambassadors-parking-project\data\datathon"+"/lor_friedrichshain_kreuzberg.geojson",
    {"amenity": ["restaurant", "cafe", "fast_food", "bar", "pub", "ice_cream", "biergarten", "food_court", "canteen"]},
    )

C:\Users\Michl\anaconda3\envs\datathon\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
parking_counts = calc_parking_spots(
    poi_lor,
    r"C:\Users\Michl\Documents\GitHub\data-ambassadors-parking-project\data\datathon"+"/parking/estimated_parking_spots_kfz.geojson",
    "sum",
    )


In [11]:
# Df with Parking lot counts
parking_counts[0]
# Df with POI + defined parking lot area
parking_counts[1]

,amenity,geometry,name,index_right,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME,BEZ,FINHALT,STAND,No. parking spots
0,restaurant,POINT (390959.231 5816831.101),Tibet Haus,12,02200211,Chamissokiez,022002,Tempelhofer Vorstadt,0220,Kreuzberg Süd,02,977115.397609,01.01.2021,133.0
1,cafe,POINT (390922.902 5816642.907),Barcomi's Café & Kaffeerösterei,12,02200211,Chamissokiez,022002,Tempelhofer Vorstadt,0220,Kreuzberg Süd,02,977115.397609,01.01.2021,6.0
2,restaurant,POINT (390915.883 5816676.379),Pagode,12,02200211,Chamissokiez,022002,Tempelhofer Vorstadt,0220,Kreuzberg Süd,02,977115.397609,01.01.2021,88.0
3,restaurant,POINT (391861.508 5816570.217),Kinnaree Thai,12,02200211,Chamissokiez,022002,Tempelhofer Vorstadt,0220,Kreuzberg Süd,02,977115.397609,01.01.2021,109.0
4,restaurant,POINT (391147.211 5816607.811),Austria,12,02200211,Chamissokiez,022002,Tempelhofer Vorstadt,0220,Kreuzberg Süd,02,977115.397609,01.01.2021,342.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,cafe,POINT (390776.013 5818589.549),Malu,33,02100102,Mehringplatz,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,624470.274067,01.01.2021,26.0
1677,restaurant,POINT (390917.367 5818246.073),Café Nullpunkt,33,02100102,Mehringplatz,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,624470.274067,01.01.2021,48.0
1678,restaurant,POINT (390840.183 5818715.297),Babami,33,02100102,Mehringplatz,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,624470.274067,01.01.2021,0.0
1679,bar,POINT (390932.194 5818487.246),John & Tommy's,33,02100102,Mehringplatz,021001,Südliche Friedrichstadt,0210,Kreuzberg Nord,02,624470.274067,01.01.2021,0.0
